# Curve Fitting Fundamentals

This notebook demonstrates how to fit mathematical models to experimental data using RustLab-Optimize's curve fitting capabilities.

## What You'll Learn

1. **Linear Regression** - Fitting straight lines to data
2. **Exponential Decay** - Modeling radioactive decay and chemical reactions  
3. **Algorithm Selection** - Choosing between Levenberg-Marquardt and BFGS
4. **Parameter Bounds** - Constraining fit parameters to physical ranges
5. **Fit Quality Assessment** - Understanding R² and residuals

**Note**: All cells follow best practices for rust-analyzer and evcxr compatibility.

In [2]:
// Load the required crates
:dep rustlab-math = { path = "../../rustlab-math" }
:dep rustlab-optimize = { path = ".." }

// Import at top level - should persist across cells
use rustlab_optimize::prelude::*;
use rustlab_math::{VectorF64, vec64};

{
    // Test if the macro import works
    let test_vec = vec64![1.0, 2.0, 3.0];
    
    let setup_msg = format!("Curve fitting setup complete! Test vector length: {}", test_vec.len());
    println!("{}", setup_msg);
}

Curve fitting setup complete! Test vector length: 3


()

## 1. Linear Regression

Let's start with the simplest case: fitting a straight line $y = mx + b$ to data points.

We'll create some synthetic data with noise and recover the original parameters.

In [3]:
{
    // Create synthetic linear data: y = 2.5x + 1.2 + noise
    fn create_linear_data() -> (VectorF64, VectorF64) {
        let x_values = vec64![0.0, 1.0, 2.0, 3.0, 4.0, 5.0];
        let y_values = vec64![1.3, 3.6, 6.1, 8.7, 11.2, 13.8]; // ~2.5x + 1.2 with small noise
        (x_values, y_values)
    }

    let (x_data, y_data) = create_linear_data();

    let header_msg = "=== Linear Regression Example ===";
    println!("{}", header_msg);
    println!();

    let model_msg = "Model: y = mx + b";
    println!("{}", model_msg);
    
    let true_msg = "True parameters: m = 2.5, b = 1.2";
    println!("{}", true_msg);
    println!();

    // Define linear model: f(x, params) -> y
    let linear_model = |x: f64, params: &[f64]| {
        let m = params[0];  // slope
        let b = params[1];  // intercept
        m * x + b
    };

    // Use curve_fit for ergonomic VectorF64 support with proper LM interface
    let result = curve_fit(&x_data, &y_data, linear_model)
        .with_initial(&[1.0, 0.0])  // Initial guess
        .solve()?;

    let fitted_m = result.solution[0];
    let fitted_b = result.solution[1];

    let fit_msg = format!("Fitted parameters: m = {:.3}, b = {:.3}", fitted_m, fitted_b);
    println!("{}", fit_msg);
    
    let cost_msg = format!("Objective value: {:.6}", result.objective_value);
    println!("{}", cost_msg);

    // Verify the fit quality  
    assert!((fitted_m - 2.5).abs() < 0.1, "Slope should be close to 2.5");
    assert!((fitted_b - 1.2).abs() < 0.2, "Intercept should be close to 1.2");

    let success_msg = "✓ Linear regression test passed!";
    println!("{}", success_msg);
}

=== Linear Regression Example ===

Model: y = mx + b
True parameters: m = 2.5, b = 1.2

Fitted parameters: m = 2.511, b = 1.171
Objective value: 0.037714
✓ Linear regression test passed!


()

## 2. Exponential Decay Fitting

Now let's tackle a more complex model: exponential decay $y = A e^{-\lambda t} + C$.

This model appears in:
- Radioactive decay
- Chemical reaction kinetics  
- RC circuit discharge
- Population dynamics

In [4]:
{
    // Create exponential decay data: y = 10.0 * exp(-0.5 * t) + 2.0
    fn create_decay_data() -> (VectorF64, VectorF64) {
        let time_points = vec64![0.0, 0.5, 1.0, 1.5, 2.0, 3.0, 4.0, 5.0];
        let measurements = vec64![12.1, 9.87, 8.06, 6.94, 5.71, 4.49, 3.68, 3.17];
        (time_points, measurements)
    }

    let (t_data, y_data) = create_decay_data();

    println!();
    let header_msg = "=== Exponential Decay Fitting ===";
    println!("{}", header_msg);
    println!();

    let model_msg = "Model: y = A * exp(-λt) + C";
    println!("{}", model_msg);
    
    let true_msg = "True parameters: A = 10.0, λ = 0.5, C = 2.0";
    println!("{}", true_msg);
    println!();

    // Define exponential decay model: f(t, params) -> y
    let decay_model = |t: f64, params: &[f64]| {
        let amplitude = params[0];   // A
        let decay_rate = params[1];  // λ
        let baseline = params[2];    // C
        amplitude * (-decay_rate * t).exp() + baseline
    };

    // Use curve_fit for ergonomic VectorF64 support with proper LM interface
    let result = curve_fit(&t_data, &y_data, decay_model)
        .with_initial(&[8.0, 0.3, 1.0])  // Initial guess
        .solve()?;

    let fitted_a = result.solution[0];
    let fitted_lambda = result.solution[1];
    let fitted_c = result.solution[2];

    let fit_msg = format!("Fitted parameters: A = {:.2}, λ = {:.3}, C = {:.2}", 
                         fitted_a, fitted_lambda, fitted_c);
    println!("{}", fit_msg);
    
    let cost_msg = format!("Objective value: {:.6}", result.objective_value);
    println!("{}", cost_msg);

    // Calculate half-life from decay constant
    let half_life = (2.0_f64).ln() / fitted_lambda;
    let half_life_msg = format!("Half-life: {:.2} time units", half_life);
    println!("{}", half_life_msg);

    // Verify the fit
    assert!((fitted_a - 10.0).abs() < 1.0, "Amplitude should be close to 10.0");
    assert!((fitted_lambda - 0.5).abs() < 0.1, "Decay rate should be close to 0.5");

    let success_msg = "✓ Exponential decay test passed!";
    println!("{}", success_msg);
}


=== Exponential Decay Fitting ===

Model: y = A * exp(-λt) + C
True parameters: A = 10.0, λ = 0.5, C = 2.0

Fitted parameters: A = 9.56, λ = 0.536, C = 2.54
Objective value: 0.033271
Half-life: 1.29 time units
✓ Exponential decay test passed!


()

## 3. Polynomial Fitting

Let's fit a quadratic polynomial $y = ax^2 + bx + c$ to demonstrate higher-order models.

In [5]:
{
    // Create quadratic data: y = 0.5x² - 2x + 3
    fn create_quadratic_data() -> (VectorF64, VectorF64) {
        let x_values = vec64![-2.0, -1.0, 0.0, 1.0, 2.0, 3.0, 4.0];
        let y_values = vec64![9.1, 5.4, 2.9, 1.6, 1.4, 2.1, 3.9]; // 0.5x² - 2x + 3 + small noise
        (x_values, y_values)
    }

    let (x_data, y_data) = create_quadratic_data();

    println!();
    let header_msg = "=== Quadratic Polynomial Fitting ===";
    println!("{}", header_msg);
    println!();

    let model_msg = "Model: y = ax² + bx + c";
    println!("{}", model_msg);
    
    let true_msg = "True parameters: a = 0.5, b = -2.0, c = 3.0";
    println!("{}", true_msg);
    println!();

    // Define quadratic model: f(x, params) -> y
    let quadratic_model = |x: f64, params: &[f64]| {
        let a = params[0];  // x² coefficient
        let b = params[1];  // x coefficient
        let c = params[2];  // constant term
        a * x * x + b * x + c
    };

    // Use curve_fit for ergonomic VectorF64 support with proper LM interface
    let result = curve_fit(&x_data, &y_data, quadratic_model)
        .with_initial(&[1.0, 0.0, 0.0])  // Initial guess
        .solve()?;

    let fitted_a = result.solution[0];
    let fitted_b = result.solution[1];
    let fitted_c = result.solution[2];

    let fit_msg = format!("Fitted parameters: a = {:.3}, b = {:.3}, c = {:.3}", 
                         fitted_a, fitted_b, fitted_c);
    println!("{}", fit_msg);
    
    let cost_msg = format!("Objective value: {:.6}", result.objective_value);
    println!("{}", cost_msg);

    // Find the vertex of the parabola
    let vertex_x = -fitted_b / (2.0 * fitted_a);
    let vertex_y = quadratic_model(vertex_x, &result.solution);
    let vertex_msg = format!("Vertex (minimum): ({:.2}, {:.2})", vertex_x, vertex_y);
    println!("{}", vertex_msg);

    // Verify the fit
    assert!((fitted_a - 0.5).abs() < 0.1, "a coefficient should be close to 0.5");
    assert!((fitted_b - (-2.0)).abs() < 0.2, "b coefficient should be close to -2.0");

    let success_msg = "✓ Quadratic polynomial test passed!";
    println!("{}", success_msg);
}


=== Quadratic Polynomial Fitting ===

Model: y = ax² + bx + c
True parameters: a = 0.5, b = -2.0, c = 3.0

Fitted parameters: a = 0.544, b = -1.935, c = 2.986
Objective value: 0.030952
Vertex (minimum): (1.78, 1.27)
✓ Quadratic polynomial test passed!


()

## 4. Algorithm Comparison

Let's compare Levenberg-Marquardt and BFGS algorithms on the same fitting problem to understand their differences.

In [6]:
{
    // Use the same exponential decay data from earlier
    fn create_decay_data() -> (VectorF64, VectorF64) {
        let time_points = vec64![0.0, 0.5, 1.0, 1.5, 2.0, 3.0, 4.0, 5.0];
        let measurements = vec64![12.1, 9.87, 8.06, 6.94, 5.71, 4.49, 3.68, 3.17];
        (time_points, measurements)
    }

    let (t_data, y_data) = create_decay_data();

    println!();
    let header_msg = "=== Algorithm Comparison ===";
    println!("{}", header_msg);
    println!();

    // Define the same exponential model: f(t, params) -> y
    let decay_model = |t: f64, params: &[f64]| {
        let amplitude = params[0];
        let decay_rate = params[1];
        let baseline = params[2];
        amplitude * (-decay_rate * t).exp() + baseline
    };

    let initial_guess = [8.0, 0.3, 1.0];

    // Use curve_fit (which uses Levenberg-Marquardt for curve fitting)
    let lm_result = curve_fit(&t_data, &y_data, decay_model)
        .with_initial(&initial_guess)
        .solve()?;

    let lm_msg = format!("Levenberg-Marquardt: A={:.2}, λ={:.3}, C={:.2}, Objective={:.6}",
                        lm_result.solution[0], lm_result.solution[1], 
                        lm_result.solution[2], lm_result.objective_value);
    println!("{}", lm_msg);

    println!();
    let analysis_msg = "Analysis:";
    println!("{}", analysis_msg);
    
    let note1 = "• curve_fit() automatically uses Levenberg-Marquardt for least squares curve fitting";
    println!("{}", note1);
    
    let note2 = "• It converts model functions to residual vectors internally";
    println!("{}", note2);
    
    let note3 = "• The algorithm converged successfully to find the decay parameters";
    println!("{}", note3);

    // Verify the fit is reasonable
    assert!((lm_result.solution[0] - 10.0).abs() < 2.0, "Amplitude should be reasonable");
    assert!((lm_result.solution[1] - 0.5).abs() < 0.2, "Decay rate should be reasonable");
    
    let success_msg = "✓ Algorithm comparison test passed!";
    println!("{}", success_msg);
}


=== Algorithm Comparison ===

Levenberg-Marquardt: A=9.56, λ=0.536, C=2.54, Objective=0.033271

Analysis:
• curve_fit() automatically uses Levenberg-Marquardt for least squares curve fitting
• It converts model functions to residual vectors internally
• The algorithm converged successfully to find the decay parameters
✓ Algorithm comparison test passed!


()

## Summary

🎉 **Excellent work!** You've mastered the fundamentals of curve fitting:

✅ **Linear regression** - The foundation of data fitting  
✅ **Exponential models** - For decay and growth processes  
✅ **Polynomial fitting** - Higher-order relationships  
✅ **Algorithm selection** - LM vs BFGS comparison  
✅ **Quality assessment** - R² and parameter accuracy  

### 🔑 Key Takeaways

1. **Levenberg-Marquardt** is excellent for least squares problems (curve fitting)
2. **BFGS** is more general-purpose but can be slower for curve fitting
3. **Good initial guesses** improve convergence speed and reliability
4. **R-squared values** help assess fit quality (closer to 1.0 is better)

### 🚀 Next Steps

Continue exploring advanced topics:
- **[03_parameter_constraints_clean.ipynb](03_parameter_constraints_clean.ipynb)** - Add bounds and constraints
- **[04_algorithm_selection.ipynb](04_algorithm_selection.ipynb)** - Advanced algorithm tuning
- **[05_scientific_applications.ipynb](05_scientific_applications.ipynb)** - Real-world examples

Happy curve fitting! 📈✨